# Introducing Drift
Now that we have set up TrustyAI to detect drifts, let's introduce some and see what happens.  
To do this, we will simply send a bunch of requests based on our testing dataset.  
Because we only sent a small size of our training dataset to TrustyAI, there is a good chance that the testing dataset will look significantly different.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import numpy as np
import requests

In [ ]:
# Disable some warning messages
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

Make sure to add your test models endpoint as `infer_endpoint` here.

In [ ]:
deployed_model_name = "jukebox"
infer_endpoint = "<paste-the-link-here>"
infer_url = f"{infer_endpoint}/v2/models/{deployed_model_name}/infer"

In [ ]:
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "dense_input",
                "shape": [1, 13],
                "datatype": "FP32",
                "data": data
            }
        ]
    }

    response = requests.post(infer_url, json=json_data, verify=False)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']


In [ ]:
# Load the scaler and test data
# TODO: Can we get these from the pipeline S3 artifact store?
with open('../2-dev_datascience/models/jukebox/1/artifacts/scaler.pkl', 'rb') as handle:
    scaler = pickle.load(handle)

X_test = pd.read_parquet('../2-dev_datascience/models/jukebox/1/artifacts/X_test.parquet')

In [ ]:
# Transform our dataset
X_test_scaled = scaler.transform(X_test)

We are sending 5000 samples from our test dataset, so this may take a few minutes.

In [ ]:
# Sending predictions
for i, sample in enumerate(X_test_scaled[:5000]):
    if i%500==0:
        print(f"{i} samples sent")
    prediction = rest_request(X_test_scaled[0].tolist())